## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/1'), creation_time=1703625444705, experiment_id='1', last_update_time=1703625444705, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/0'), creation_time=1703625251801, experiment_id='0', last_update_time=1703625251801, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'file:///C:/Users/AIDOO K. ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/1/4da17d4f29074aeca5872ad54e302418/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/1'), creation_time=1703625444705, experiment_id='1', last_update_time=1703625444705, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/0'), creation_time=1703625251801, experiment_id='0', last_update_time=1703625251801, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1703629185093, description='', last_updated_timestamp=1703629185093, latest_versions=[], name='iris-classifier', tags={}>]

In [23]:
run_id = client.search_runs(experiment_ids='1')[0]
print(run_id)

<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "4da17d4f29074aeca5872ad54e302418", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2023-12-27 00:32:27.777045", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.18", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"fcedf0d33b0

In [29]:
runs = mlflow.search_runs(experiment_ids=['1'])
if not runs.empty:
    run_id = runs.iloc[0]['run_id']
    print(run_id)

4da17d4f29074aeca5872ad54e302418


In [30]:


mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2023/12/27 12:14:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1703679270658, current_stage='None', description='', last_updated_timestamp=1703679270658, name='iris-classifier', run_id='4da17d4f29074aeca5872ad54e302418', run_link='', source=('file:///C:/Users/AIDOO K. '
 'ENOCH/Documents/GitHub/RGT/MLOPS/mlflow_examples/artifacts_local/1/4da17d4f29074aeca5872ad54e302418/artifacts/models'), status='READY', status_message='', tags={}, user_id='', version='1'>

In [20]:
runs = mlflow.search_runs(experiment_ids='1', filter_string='')
